In [1]:
import os
import jax 
import numpy as np
from jax import random
import jax.numpy as jnp
from jax import make_jaxpr
from functools import partial
from jax import grad, jit, vmap
import matplotlib.pyplot as plt
from jax.config import config
from scipy.integrate import odeint
config.update("jax_enable_x64", True) # enable 64-bit precision
config.update("jax_platform_name", "cpu") # "gpu"  uncomment this line to run on CPU
os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=8' 

In [30]:
USE_TPU = False
if USE_TPU:
    import jax.tools.colab_tpu
    jax.tools.colab_tpu.setup_tpu()

In [2]:
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.xmargin'] = 0


In [3]:
def whereis(x):
    print(x.device_buffer.device())

In [4]:
x = jnp.array([1, 2, 3])
whereis(x)

TFRT_CPU_0


### [VMAP](https://jiayiwu.me/blog/2021/04/05/learning-about-jax-axes-in-vmap.html)

- [YouTube](https://www.youtube.com/watch?v=W1vfBDFLm7Q)

In [5]:
a = jnp.array(([1, 3], [2, 4]))
b = jnp.array(([11, 3], [12, 40]))
a, b

(Array([[1, 3],
        [2, 4]], dtype=int64),
 Array([[11,  3],
        [12, 40]], dtype=int64))

In [6]:
jnp.add(a, b)

Array([[12,  6],
       [14, 44]], dtype=int64)

In [7]:
vmap(jnp.add, in_axes=(0,0), out_axes=0)(a, b)
# row of a + corresponding row of b, then stack the results by rows
# same as np.add(a, b)

Array([[12,  6],
       [14, 44]], dtype=int64)

In [8]:
vmap(jnp.add, in_axes=(0,0), out_axes=1)(a, b)
# row of a + corresponding row of b, then stack the results by columns
# same as np.add(a, b).T

Array([[12, 14],
       [ 6, 44]], dtype=int64)

In [9]:
vmap(jnp.add, in_axes=(0,1), out_axes=0)(a, b)
# row of a + column of b, then stack the results by rows
# same as np.add(a, b.T)

Array([[12, 15],
       [ 5, 44]], dtype=int64)

In [10]:
vmap(jnp.add, in_axes=(0,1), out_axes=1)(a, b)
# row of a + column of b, then stack the results by columns

Array([[12,  5],
       [15, 44]], dtype=int64)

In [11]:
vmap(jnp.add, in_axes=(1,0), out_axes=0)(a, b)
# column of a + row of b, then stack the results by rows
# same as np.add(a.T, b)

Array([[12,  5],
       [15, 44]], dtype=int64)

In [12]:
a = np.arange(6).reshape(2, 3) + 10
b = 2 
c = np.arange(6).reshape(2, 3) + 20

def f(a, b, c):
    return a + b + c

print(a, b, c, sep='\n')

v_f = jit(vmap(f, in_axes=(0, None, 0)))

make_jaxpr(v_f)(a, b, c)

[[10 11 12]
 [13 14 15]]
2
[[20 21 22]
 [23 24 25]]


{ lambda ; a:i64[2,3] b:i64[] c:i64[2,3]. let
    d:i64[2,3] = xla_call[
      call_jaxpr={ lambda ; e:i64[2,3] f:i64[] g:i64[2,3]. let
          h:i64[] = convert_element_type[new_dtype=int64 weak_type=False] f
          i:i64[2,3] = add e h
          j:i64[2,3] = add i g
        in (j,) }
      name=f
    ] a b c
  in (d,) }

In [13]:
jax.devices()

[CpuDevice(id=0),
 CpuDevice(id=1),
 CpuDevice(id=2),
 CpuDevice(id=3),
 CpuDevice(id=4),
 CpuDevice(id=5),
 CpuDevice(id=6),
 CpuDevice(id=7)]

In [15]:
from jax import pmap 

p_f = pmap(f, in_axes=(0, None, 0))
a = np.random.random(size=(8,3))
b = 4
c = np.random.random(size=(8,3))

p_f(a, b, c)

Array([[4.46896951, 4.60178612, 5.39223538],
       [4.5737435 , 5.7597772 , 4.04448616],
       [5.32997673, 5.27548329, 4.98168802],
       [5.02639403, 4.65515707, 4.79183542],
       [4.63106177, 5.10006782, 4.83879445],
       [5.12578873, 4.91460895, 4.61827822],
       [5.05777006, 5.59822285, 4.9888262 ],
       [5.27865684, 5.06300719, 4.89047318]], dtype=float64)

- composition

In [18]:
def f(a, b, c):
    return a @ b + c

a = np.random.random(size=(2,3))
b = np.random.random(size=(3,4))
c = np.random.random(size=(2,4))

f(a, b, c).shape

(2, 4)

In [23]:
v_f = vmap(f)

a = np.random.random(size=(5, 2, 3))
b = np.random.random(size=(5, 3, 4))
c = np.random.random(size=(5, 2, 4))

v_f(a, b, c).shape

(5, 2, 4)

In [24]:
make_jaxpr(v_f)(a, b, c)

{ lambda ; a:f64[5,2,3] b:f64[5,3,4] c:f64[5,2,4]. let
    d:f64[5,2,4] = dot_general[
      dimension_numbers=(((2,), (1,)), ((0,), (0,)))
      precision=None
      preferred_element_type=None
    ] a b
    e:f64[5,2,4] = add d c
  in (e,) }

In [25]:
jv_f = jit(vmap(f))

a = np.random.random(size=(5, 2, 3))
b = np.random.random(size=(5, 3, 4))
c = np.random.random(size=(5, 2, 4))

print(v_f(a, b, c).shape)
make_jaxpr(jv_f)(a, b, c)

(5, 2, 4)


{ lambda ; a:f64[5,2,3] b:f64[5,3,4] c:f64[5,2,4]. let
    d:f64[5,2,4] = xla_call[
      call_jaxpr={ lambda ; e:f64[5,2,3] f:f64[5,3,4] g:f64[5,2,4]. let
          h:f64[5,2,4] = dot_general[
            dimension_numbers=(((2,), (1,)), ((0,), (0,)))
            precision=None
            preferred_element_type=None
          ] e f
          i:f64[5,2,4] = add h g
        in (i,) }
      name=f
    ] a b c
  in (d,) }

In [27]:
pv_f = pmap(v_f)

a = np.random.random(size=(8, 5, 2, 3))
b = np.random.random(size=(8, 5, 3, 4))
c = np.random.random(size=(8, 5, 2, 4))

print(v_f(a, b, c).shape)
make_jaxpr(pv_f)(a, b, c)

(8, 5, 2, 4)


{ lambda ; a:f64[8,5,2,3] b:f64[8,5,3,4] c:f64[8,5,2,4]. let
    d:f64[8,5,2,4] = xla_pmap[
      axis_name=<axis 0x7fb1381e9160>
      axis_size=8
      backend=None
      call_jaxpr={ lambda ; e:f64[5,2,3] f:f64[5,3,4] g:f64[5,2,4]. let
          h:f64[5,2,4] = dot_general[
            dimension_numbers=(((2,), (1,)), ((0,), (0,)))
            precision=None
            preferred_element_type=None
          ] e f
          i:f64[5,2,4] = add h g
        in (i,) }
      devices=None
      donated_invars=(False, False, False)
      global_arg_shapes=(None, None, None)
      global_axis_size=None
      in_axes=(0, 0, 0)
      name=f
      out_axes=(0,)
    ] a b c
  in (d,) }

In [28]:
key = random.PRNGKey(0)
print(jax.random.uniform(key, (3,)))
print(jax.random.uniform(key, (3,)))

[0.57450053 0.09968609 0.74196595]
[0.57450053 0.09968609 0.74196595]


- RANDOM

In [29]:
key = jax.random.PRNGKey(124)
for _ in range(5):
    key, subkey = jax.random.split(key)
    print(jax.random.uniform(subkey))


0.6773641507104642
0.0482590299181076
0.42465834799117474
0.0919238766178283
0.7752573371832019
